In [1]:
import pandas as pd

import requests
import json

from pprint import pprint

import sqlite3

import time

import config

In [2]:
## Static

API_KEY = config.API_KEY
BASE_URL = 'http://api.steampowered.com/'
status = 200

In [3]:
def steamFriends(steamid):
    
    time.sleep(2)
    ## Establish the URL
    friend_url = ''.join([BASE_URL,
        'ISteamUser/GetFriendList/v0001/?key=',API_KEY,
        '&steamid=',str(steamid),
        '&relationship=friend'])
    
    ## Setting connection to DB
    conn = sqlite3.connect('steam.db')
    c = conn.cursor()
    
    ## Pull from using API
    r = requests.get(friend_url)
    
    if r.status_code != 200:
        return r.status_code
    
    content = json.loads(r.content)
    
    if 'friendslist' not in content:
        return 200
    
    for i in content['friendslist']['friends']:
        sid = int(i['steamid'])
        
        c.execute('SELECT * FROM parse_id WHERE id = ? UNION SELECT * FROM finished_id WHERE id = ?', (sid,sid))
        rows = c.fetchall()
        
        if len(rows) == 0:
            insert = "INSERT INTO parse_id VALUES (" + str(sid) + ")"
            c.execute(insert)
            
        else:
            continue
            
    conn.commit()
    
    conn.close()
    return 200

In [14]:
def steamGames(steamid):
    
    time.sleep(2)
    ## Establish the URL
    game_url = ''.join([BASE_URL, 
                      'IPlayerService/GetOwnedGames/v0001/?key=',API_KEY,
                      '&steamid=',str(steamid),
                      '&format=json'])
    
    ## Setting connection to DB
    conn = sqlite3.connect('steam.db')
    c = conn.cursor()
    
    ## Pulling from API
    r = requests.get(game_url)
    
    if r.status_code != 200:
        return r.status_code
    
    content = json.loads(r.content)
    
    if content['response'] == {}:
        return 200
    
    games = content['response']['games']
    for g in games:
        appid = g['appid']
        playtime = g['playtime_forever']
        if playtime == 0:
            continue
        
        row = [steamid, appid, playtime]
        
        c.execute("INSERT INTO gameplay (steamID, appID, playtime) VALUES (?,?,?);", row)
        
    conn.commit()
    
    conn.close()
    
    return 200

In [20]:
steamid = 76561198064781699
game_url = ''.join([BASE_URL, 
                      'IPlayerService/GetOwnedGames/v0001/?key=',API_KEY,
                      '&steamid=',str(steamid),
                      '&format=json'])

r = requests.get(game_url)

In [21]:
r.status_code

200

In [22]:
content = json.loads(r.content)

In [29]:
print(content)

{'response': {'game_count': 76, 'games': [{'appid': 400, 'playtime_forever': 23, 'playtime_windows_forever': 0, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0}, {'appid': 70100, 'playtime_forever': 339, 'playtime_windows_forever': 0, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0}, {'appid': 47810, 'playtime_forever': 0, 'playtime_windows_forever': 0, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0}, {'appid': 620, 'playtime_forever': 0, 'playtime_windows_forever': 0, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0}, {'appid': 205790, 'playtime_forever': 0, 'playtime_windows_forever': 0, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0}, {'appid': 204360, 'playtime_forever': 142, 'playtime_windows_forever': 0, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0}, {'appid': 230410, 'playtime_forever': 0, 'playtime_windows_forever': 0, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0}, {'appid': 239030, 'playtime_forever': 83, 'playtime_window

In [24]:
if content['response'] == {}:
    print('yeet')

In [28]:
if 'games' in content['response']:
    print('yeeter')

yeeter


In [15]:
status = 200

while status == 200:
    time.sleep(2)
    conn = sqlite3.connect('steam.db')
    c = conn.cursor()
    
    ## Pull ID
    c.execute("SELECT id FROM parse_id LIMIT 1")
    steamID = c.fetchone()[0]
    
    ## Insert ID
    check = "SELECT * FROM finished_id WHERE id = "+str(steamID)
    c.execute(check)
    rows = c.fetchall()
    
    if len(rows) == 0:
        insert = 'INSERT INTO finished_id VALUES ('+str(steamID)+')'
        c.execute(insert)
        
    ## Delete ID
    delete = 'DELETE FROM parse_id WHERE id = '+str(steamID)
    c.execute(delete)
    
    conn.commit()
    
    conn.close()
    
    status = steamFriends(steamID)
    
    status = steamGames(steamID)
    
    if status != 200:
        print(status)

KeyboardInterrupt: 

In [5]:
conn = sqlite3.connect('steam.db')
c = conn.cursor()

sql = "SELECT id FROM finished_id limit 10"
c.execute(sql)

rows = c.fetchall()

print(rows)

[(76561197994871291,), (76561198011799158,), (76561198104062560,), (76561198015016293,), (76561197964165126,), (76561197960435530,), (76561197968292158,), (76561198022003287,), (76561197970953718,), (76561197970858434,)]


In [ ]:
    conn = sqlite3.connect('steam.db')
    c = conn.cursor()
    
    ## Pull ID
    c.execute("SELECT id FROM parse_id LIMIT 1")
    steamID = c.fetchone()[0]
    
    ## Insert ID
    check = "SELECT * FROM finished_id WHERE id = "+str(steamID)
    c.execute(check)
    rows = c.fetchall()
    
    if len(rows) == 0:
        insert = 'INSERT INTO finished_id VALUES ('+str(steamID)+')'
        c.execute(insert)
        
    ## Delete ID
    delete = 'DELETE FROM parse_id WHERE id = '+str(steamID)
    c.execute(delete)